In [15]:
from DrissionPage import ChromiumOptions
path = r'C:\Program Files\Google\Chrome\Application\chrome.exe'
ChromiumOptions().set_browser_path(path).save()

配置已保存到文件: C:\Users\II\PycharmProjects\djangoProject\djangoProject\biyesheji\venv\lib\site-packages\DrissionPage\_configs\configs.ini
以后程序可自动从文件加载配置


'C:\\Users\\II\\PycharmProjects\\djangoProject\\djangoProject\\biyesheji\\venv\\lib\\site-packages\\DrissionPage\\_configs\\configs.ini'

In [5]:
from DrissionPage import ChromiumPage
from DrissionPage.common import Keys
import csv
import time

dp = ChromiumPage()
dp.get('https://www.bilibili.com/')
dp.ele('@class=channel-link').click()


<ChromiumElement a class='channel-link' href='https://www.bilibili.com/anime/?spm_id_from=333.1007.0.0' style='letter-spacing:2px;' target='_blank'>

In [2]:
from DrissionPage import ChromiumPage
from DrissionPage.common import Keys
import time

dp = ChromiumPage()
dp.get('https://www.jd.com/')
dp.ele('#key').input('内存')
dp.ele('#key').input(Keys.ENTER)
time.sleep(3)  # 确保搜索完成

for page in range(1, 11):
    # 获取当前页所有商品
    products = dp.eles('li.gl-item')
    if not products:
        break

    for product in products:
        # 获取商品链接并打开新标签页
        product_link = product.ele('tag:a')
        tab = dp.new_tab(product_link.link, activate=True)  # 激活新标签页
        tab.wait.load_start()  # 等待新页面加载

        try:
            # 获取店铺名称
            shopname = tab.ele('.p-shop').text
            print(f"店铺名: {shopname}")

            # 处理规格1
            spec1_wrapper = tab.ele('#choose-attr-1')
            spec1_options = spec1_wrapper.eles('dd a')
            for spec1 in spec1_options:
                spec1.click()
                time.sleep(1)  # 等待规格1加载
                tab.wait.ele_loaded('#choose-attr-2', timeout=3)  # 显式等待规格2区域加载

                # 处理规格2
                spec2_wrapper = tab.ele('#choose-attr-2')
                spec2_options = spec2_wrapper.eles('dd a')
                for spec2 in spec2_options:
                    spec2.click()
                    time.sleep(1)
                    # 获取价格
                    price = tab.ele('.price').text
                    print(f"规格组合: {spec1.text} + {spec2.text}, 价格: {price}")

        except Exception as e:
            print(f"处理商品时出错: {e}")

        finally:
            tab.close()  # 关闭当前标签页
            dp.wait(1)
            dp.set.active_tab(dp.latest_tab)  # 切回主标签页

    # 翻页操作
    next_btn = dp.ele('.pn-next', timeout=3)
    if next_btn:
        next_btn.click()
        time.sleep(3)  # 等待新页加载
    else:
        break